In [3]:
# from transformers import BertModel
# from tokenization_kobert import KoBertTokenizer
# MODEL_NAME = "monologg/kobert"
# model = BertModel.from_pretrainead(MODEL_NAME)
# kobert_tokenizer = KoBertTokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [1]:
from transformers import ElectraModel, ElectraTokenizer, AutoTokenizer

model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")
# tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", use_fast=True)

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# "Kobert"와 "KoElectra" 간 tokenizing 결과는 유사하다
# QnA 성능이 더 좋은 "KoElectra"로 실험

# print('KoBert tokenizer:')
# print(
# kobert_tokenizer.tokenize("천둥번개를 동반한 소나기가 34번 내렸습니다. 상당히 많이 내렸는데"),'\n',
# kobert_tokenizer.tokenize("장갑을 낀 사람이 밭에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다"))
# print('\n')
print('KoElectra tokenizer:')
print(
tokenizer.tokenize("천둥번개를 동반한 소나기가 34번 내렸습니다. 상당히 많이 내렸는데"),'\n',
tokenizer.tokenize("장갑을 낀 사람이 밭에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다"))

KoElectra tokenizer:
['천둥', '##번', '##개', '##를', '동반', '##한', '소나기', '##가', '34', '##번', '내렸', '##습', '##니다', '.', '상당히', '많이', '내렸', '##는데'] 
 ['장갑', '##을', '낀', '사람', '##이', '밭', '##에', '있', '##다', '그', '사람', '##이', '허리', '##를', '숙이', '##고', '있', '##다', '그', '사람', '##이', '작업', '##을', '한다']


In [ ]:
# pororo로 postagging 후 명사에 masking 추가 + rnn보다 강건한 transformer 계열 모델 사용

# from pororo import Pororo
# pos = Pororo(task="pos", lang="ko")
# sent = "검은색 옷을 입은 사람이 들판에 있다\n그 사람이 설명을 한다\n그 사람이 두 손을 펼친다"
# pos(sent)

In [161]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator", use_fast=True)

In [36]:
# unused vocab 확인
unused_tok2idx = {k:v for k,v in tokenizer.get_vocab().items() if 'unused' in k}
len(unused_tok2idx), min(unused_tok2idx.values()), max(unused_tok2idx.values())

(1000, 34000, 34999)

In [9]:
"""
<KoElectra.Tokenizer, Mecab 을 이용한 vocab matrix 생성>

matrix = np.zeros((vocab, 768))

- 목적 : 정답단어 분절X, unk 어휘는 일단 무시

1. 정답 단어 중 분절 단어 모으기

add_tokens = [new1, new2...]
num_added_tokens = Tokenizer.add_tokens(add_tokens) # 34000 ~

vocab = Tokenizer.vocab
i2w = {idx:k for w, idx in vocab}

vector1 = get_word_emb
vector2 = get_new_emb

answers embedding:
- 
"""
answers = [
      "눈이라고 말함",
      "우박이라고 말함",
      "바람이라고 말함",
      "소나기라고 말함",
      "구름이라고 말함"
      "설원이라고 말함"
    ]

new_tokens = []

for answer in answers:

    for morph in mecab(answer):
        
        answer_tokened = tokenizer.tokenize(morph)

        if len(answer_tokened)==1 or morph in new_tokens:
            continue

        # subword 분절 여부 check -> 해당 단어 저장
        print(morph, answer_tokened)
        new_tokens.append(morph)

new_tokens
added_num = tokenizer.add_tokens(new_tokens)
added_num

우박 ['우', '##박']
설원 ['설', '##원']


2

In [ ]:
from tqdm import tqdm

embsize = model.config.embedding_size
init_vocab_cnt = len(tokenizer)
final_vocab_cnt = init_vocab_cnt +len(new_tokens)

matrix = np.zeros((final_vocab_cnt, embsize))
w2idx = sorted(tokenizer.vocab.items(), key=lambda x: x[1])
w2idx += [(w, idx) for idx, w in enumerate(new_tokens, init_vocab_cnt)]

vocab_words = [w for w, _ in w2idx]

batch_size = 3000
for i in tqdm(range(0, init_vocab_cnt, batch_size)):
    word_list = vocab_words[i:min(i+batch_size, init_vocab_cnt)]
    wordvecs = get_word_vector_batch(word_list, tokenizer, model, 4)
    if i==0:
        print(wordvecs.size())
    matrix[i:min(i+batch_size, init_vocab_cnt), :] = wordvecs.detach().numpy()


In [346]:
sents = ['우박', '설원']
pool_method = 4
encoded_batch = tokenizer.batch_encode_plus(sents, return_tensors="pt",padding=True)
output = get_hidden_states(encoded_batch, model)
output = output[pool_method-1]

token_ids_word_li = []
for i in range(len(sents)):
    idx = get_word_idx(sents[i], words[i])
    token_ids_word = np.where(np.array(encoded_batch.word_ids(i)) == idx)
    token_ids_word_li.append(token_ids_word)
# Only select the tokens that constitute the requested word
word_tokens_outputs = []
for i in range(len(token_ids_word_li)):
    word_tokens_output = output[i, token_ids_word_li[i][0],:].mean(dim=0).squeeze()
    word_tokens_outputs.append(word_tokens_output)
word_tokens_outputs = torch.stack(word_tokens_outputs)


In [270]:
import json
vocab_save_fname = f'./data/video-narr/tokenizer_vocab_{final_vocab_cnt}_{embsize}.json'
print("Write into %s" % vocab_save_fname)

with open(vocab_save_fname, 'w') as file:
    json.dump(tokenizer.vocab, file, indent=4)

Write into ./data/video-narr/tokenizer_vocab_35002_768.json


In [37]:
import json
vocab_save_fname = f'./tokenizer_vocab_{final_vocab_cnt}_{embsize}.json'
print("Write into %s" % vocab_save_fname)
tokenizer.save_vocabulary('./')

Write into ./tokenizer_vocab_35004_768.json


('./vocab.txt',)

In [44]:
sorted([(k,v) for k,v in tokenizer.vocab.items()], key = lambda x : x[1])

[('[PAD]', 0),
 ('[UNK]', 1),
 ('[CLS]', 2),
 ('[SEP]', 3),
 ('[MASK]', 4),
 ('!', 5),
 ('"', 6),
 ('#', 7),
 ('$', 8),
 ('%', 9),
 ('&', 10),
 ("'", 11),
 ('(', 12),
 (')', 13),
 ('*', 14),
 ('+', 15),
 (',', 16),
 ('-', 17),
 ('.', 18),
 ('/', 19),
 ('0', 20),
 ('1', 21),
 ('2', 22),
 ('3', 23),
 ('4', 24),
 ('5', 25),
 ('6', 26),
 ('7', 27),
 ('8', 28),
 ('9', 29),
 (':', 30),
 (';', 31),
 ('<', 32),
 ('=', 33),
 ('>', 34),
 ('?', 35),
 ('@', 36),
 ('A', 37),
 ('B', 38),
 ('C', 39),
 ('D', 40),
 ('E', 41),
 ('F', 42),
 ('G', 43),
 ('H', 44),
 ('I', 45),
 ('J', 46),
 ('K', 47),
 ('L', 48),
 ('M', 49),
 ('N', 50),
 ('O', 51),
 ('P', 52),
 ('Q', 53),
 ('R', 54),
 ('S', 55),
 ('T', 56),
 ('U', 57),
 ('V', 58),
 ('W', 59),
 ('X', 60),
 ('Y', 61),
 ('Z', 62),
 ('[', 63),
 ('\\', 64),
 (']', 65),
 ('^', 66),
 ('_', 67),
 ('`', 68),
 ('a', 69),
 ('b', 70),
 ('c', 71),
 ('d', 72),
 ('e', 73),
 ('f', 74),
 ('g', 75),
 ('h', 76),
 ('i', 77),
 ('j', 78),
 ('k', 79),
 ('l', 80),
 ('m', 81),
 ('n

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./data/video-narr/tokenizer/')

In [10]:
cnt = len(tokenizer)
print(cnt, added_num)
# tokenizer.save(f'./data/video-narr/tokenizer_vocab_{cnt}_{embsize}.json', True)

35002 0


In [72]:
# tokenize 개수가 같아도 subword 불일치
tokenizer.tokenize('우박이라고 말함'), mecab('우박이라고 말함')

(['우', '##박이', '##라고', '말', '##함'], ['우박', '이', '라고', '말', '함'])

In [ ]:
"소나기 vs 구름, 작년"

In [352]:
mecab('자전거를 들이받음'), tokenizer.tokenize('들이받')

(['자전거', '를', '들이받', '음'], ['들이', '##받'])

In [16]:
tokenizer.tokenize('안녕 내 사랑', add_special_tokens=True)

['[CLS]', '안녕', '내', '사랑', '[SEP]']

In [11]:
final_vocab_cnt = len(tokenizer)
embsize = model.config.embedding_size
vocab_save_fname = f'./data/video-narr/tokenizer_vocab_{final_vocab_cnt}_{embsize}'
save_dir = './data/video-narr/tokenizer/'
tokenizer.save_pretrained(save_dir)

('./data/video-narr/tokenizer/tokenizer_config.json',
 './data/video-narr/tokenizer/special_tokens_map.json',
 './data/video-narr/tokenizer/vocab.txt',
 './data/video-narr/tokenizer/added_tokens.json',
 './data/video-narr/tokenizer/tokenizer.json')

In [13]:
tokenizer_loaded = AutoTokenizer.from_pretrained('./data/video-narr/tokenizer/', use_fast=True)
len(tokenizer_loaded)

35002

In [7]:
import torch
from konlpy.tag import Mecab
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def todevice(device, tokenized):
    return {k:v.to(device) for k,v in tokenized.items()}

mecab = Mecab().morphs
a = '장갑을 낀 사람이 밭에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다'
# Mecab().pos("보호대")

def get_hidden_states(encoded, model):
    """Push input IDs through model. Stack and sum `layers` (last four by default).
    Select only those subword token outputs that belong to our word of interest
    and average them."""
    
    # with torch.no_grad():
    #     output = model(**encoded, output_hidden_states = True)
    
    model.eval()
    model.to(device)
    if torch.cuda.device_count() > 1:
        model = torch.nn.DataParallel(model)
    
    last_hidden_state, hidden_states, attentions = model(**todevice(device, encoded),
                                                                    output_hidden_states = True, 
                                                                    output_attentions=True,
                                                                    return_dict=False) # return_dict=True 시 outputs = model(**pt)로 return 받아야

    # # 512 words
    # print('last hidden state shape:', last_hidden_state.shape)

    # # 13:(initial embeddings + 12 BERT layers)
    # print('hidden state shape:', torch.stack(hidden_states).size()) # (layer, N, T, D)

    # print('attentions shape:', torch.stack(attentions).size())

    # Remove batch-dim, Swap dimensions 0 and 1.
    token_embeddings = torch.stack(hidden_states).permute(1,2,0,3) # (N, T, layer, D)

    #https://is-rajapaksha.medium.com/bert-word-embeddings-deep-dive-32f6214f02bf

    # initial embeddings can be taken from 0th layer of hidden states
    # hidden_states = (maxlen, layers, dimension)
    word_embed_2 = token_embeddings[:,:,0,:] # [N, T, L, D] -> [N, T, D]
    # sum of all hidden states
    word_embed_3 = token_embeddings.sum(2) # [N, T, L, D] -> [N, T, D]
    # sum of second to last layer
    word_embed_4 = token_embeddings[:,:,2:,:].sum(2) # [N, T, 13, D] -> [N, T, 11, D] -> [N, T, D]
    # sum of last four layer
    word_embed_5 = token_embeddings[:,:,-4:,:].sum(2) # [N, T, 4, D] -> [N, T, D]
    # concatenate last four layers
    word_embed_6 = torch.cat([token_embeddings[:,:,i,:] for i in range(-4, -1+1)], dim=1) # [N, T, 13, D] -> [N, T, 4, D] -> [N, T, 4*D]
    return word_embed_2, word_embed_3, word_embed_4, word_embed_5, word_embed_6

    # # Get all hidden states
    # states = output.hidden_states # 13, 1, 512, 768

    # # Stack and sum all requested layers
    # output = torch.stack([states[i] for i in layers]).sum(0).squeeze()

    # return output

def get_word_idx(sent: str, word: str):
    return mecab(sent).index(word)

def get_orgword_vector(sent, word, vocab_idx, tokenizer, model, pool_method:int):
    # sent : tokenized and joined with space; ex. "나 는 학교 에 간 다" or "학교"
    # sent = ' '.join(mecab('우박이라고함'))
    
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")

    if vocab_idx > len(tokenizer):
        idx = get_word_idx(sent, word)
        token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    else:
         token_ids_word = (np.array([1]),) # [CLS] token

    output = get_hidden_states(encoded, model)
    output = output[pool_method-1]

    # Only select the tokens that constitute the requested word
    word_tokens_output = output[token_ids_word[0]].squeeze()
    return word_tokens_output if len(token_ids_word[0])==1 else word_tokens_output.mean(dim=0)

def get_orgword_vector_batch(sents, words, tokenizer, model, pool_method:int):
    # sent : tokenized and joined with space; ex. "나 는 학교 에 간 다" or "학교"
    # sent = ' '.join(mecab('우박이라고함'))
    
    encoded_batch = tokenizer.batch_encode_plus(sents, return_tensors="pt",padding=True)
    output = get_hidden_states(encoded_batch, model)
    output = output[pool_method-1]
    
    token_ids_word_li = []
    for i in range(len(sents)):
        idx = get_word_idx(sents[i], words[i])
        token_ids_word = np.where(np.array(encoded_batch.word_ids(i)) == idx)
        token_ids_word_li.append(token_ids_word)
        
    # Only select the tokens that constitute the requested word
    word_tokens_outputs = []
    for i in range(len(token_ids_word_li)):
        word_tokens_output = output[i, token_ids_word_li[i][0],:].mean(dim=0).squeeze()
        word_tokens_outputs.append(word_tokens_output)
    word_tokens_outputs = torch.stack(word_tokens_outputs)
    return word_tokens_outputs

def get_word_vector_batch(words, tokenizer, model, pool_method:int):
    # sent : tokenized and joined with space; ex. "나 는 학교 에 간 다" or "학교"
    # sent = ' '.join(mecab('우박이라고함'))
    
    encoded_batch = tokenizer.batch_encode_plus(words, return_tensors="pt", padding=True)
    output = get_hidden_states(encoded_batch, model)


    output = output[pool_method-1]
    token_ids_word = (np.array([1]),) # [CLS] token

    # Only select the tokens that constitute the requested word
    word_tokens_output = output[:,token_ids_word,:].squeeze()
    return word_tokens_output

# layers = [-4, -3, -2, -1]

# sent = "장갑을 낀 사람이 설원에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다"
# word = '설원'

# for answer in ["밭", "바다", "갯벌", "설원", "계곡"]:
#     idx = get_word_idx(sent, answer)
#     word_embedding = get_orgword_vector(sent, idx, tokenizer, model, layers, 1)


In [ ]:
a = '장갑을 낀 사람이 밭에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다'
b = '장갑'

tokenizer(
                b,
                max_length = tokenizer.model_max_length,
                return_tensors="pt",
                truncation = False,
                add_special_tokens = True,
                padding = 'max_length'
                )

In [14]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def todevice(device, tokenized):
    return {k:v.to(device) for k,v in tokenized.items()}

a = '장갑을 낀 사람이 밭에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다'
b = '장갑'

def get_word_emb(sent):
    tokenized = tokenizer(
                sent,
                max_length = tokenizer.model_max_length,
                return_tensors="pt",
                truncation = False,
                add_special_tokens = True,
                padding = 'max_length'
                )

    model.eval()
    model.to(device)
    last_hidden_state, hidden_states, attentions = model(**todevice(device, tokenized),
                                                                    output_hidden_states = True, 
                                                                    output_attentions=True,
                                                                    return_dict=False) # return_dict=True 시 outputs = model(**pt)로 return 받아야
    # 512 words
    print('last hidden state shape:', last_hidden_state.shape)

    # 13:(initial embeddings + 12 BERT layers)
    print('hidden state shape:', torch.stack(hidden_states).size()) 

    print('attentions shape:', torch.stack(attentions).size())

    # Remove batch-dim, Swap dimensions 0 and 1.
    token_embeddings = torch.stack(hidden_states).squeeze().permute(1,0,2)
    token_embeddings.size() # [512, 13, 768] (maxlen, layer, dimension)

    #https://is-rajapaksha.medium.com/bert-word-embeddings-deep-dive-32f6214f02bf

    # initial embeddings can be taken from 0th layer of hidden states
    # hidden_states = (vocabs, layers, dimension)

    word_embed_2 = token_embeddings[:,0,:] # [512, 13, 768] -> [512, 768]
    # sum of all hidden states
    word_embed_3 = token_embeddings.sum(1) # [512, 13, 768] -> [512, 768]
    # sum of second to last layer
    word_embed_4 = token_embeddings[:,2:,:].sum(1) # [512, 13, 768] -> [512, 11, 768] -> [512, 768]
    # sum of last four layer
    word_embed_5 = token_embeddings[:,-4:,:].sum(1) # [512, 4, 768] -> [512, 768]
    # concatenate last four layers
    word_embed_6 = torch.cat([token_embeddings[:,i,:] for i in range(-4, -1+1)], dim=1) # [512, 13, 768] -> [512, 4, 768] -> [512, 4*768]
    return word_embed_2, word_embed_3, word_embed_4, word_embed_5, word_embed_6

token_embeddings_a = get_word_emb(a)[0]
token_embeddings_b = get_word_emb(b)[0]

#token_embeddings_a[0]==token_embeddings_b[0]

last hidden state shape: torch.Size([1, 512, 768])
hidden state shape: torch.Size([13, 1, 512, 768])
attentions shape: torch.Size([12, 1, 12, 512, 512])
last hidden state shape: torch.Size([1, 512, 768])
hidden state shape: torch.Size([13, 1, 512, 768])
attentions shape: torch.Size([12, 1, 12, 512, 512])


In [ ]:
sentence_embedding = torch.mean(token_vecs, dim=0)

In [43]:
# Remove batch-dim, Swap dimensions 0 and 1.
token_embeddings = torch.stack(hidden_states).squeeze().permute(1,0,2)
token_embeddings.size() # [512, 13, 768] (maxlen, layer, dimension)

#https://is-rajapaksha.medium.com/bert-word-embeddings-deep-dive-32f6214f02bf

# initial embeddings can be taken from 0th layer of hidden states
# hidden_states = (vocabs, layers, dimension)

word_embed_2 = token_embeddings[:,0,:] # [512, 13, 768] -> [512, 768]
# sum of all hidden states
word_embed_3 = token_embeddings.sum(1) # [512, 13, 768] -> [512, 768]
# sum of second to last layer
word_embed_4 = token_embeddings[:,2:,:].sum(1) # [512, 13, 768] -> [512, 11, 768] -> [512, 768]
# sum of last four layer
word_embed_5 = token_embeddings[:,-4:,:].sum(1) # [512, 4, 768] -> [512, 768]
# concatenate last four layers
word_embed_6 = torch.cat([token_embeddings[:,i,:] for i in range(-4, -1+1)], dim=1) # [512, 13, 768] -> [512, 4, 768] -> [512, 4*768]
print(
word_embed_2.size(),
word_embed_3.size(),
word_embed_4.size(),
word_embed_5.size(),
word_embed_6.size(),
)

torch.Size([512, 768]) torch.Size([512, 768]) torch.Size([512, 768]) torch.Size([512, 768]) torch.Size([512, 3072])


In [ ]:

# Calculate the average of all 512 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [47]:
tokenizer.tokenize("장갑을 낀 사람이 밭에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다")

['장갑',
 '##을',
 '낀',
 '사람',
 '##이',
 '밭',
 '##에',
 '있',
 '##다',
 '그',
 '사람',
 '##이',
 '허리',
 '##를',
 '숙이',
 '##고',
 '있',
 '##다',
 '그',
 '사람',
 '##이',
 '작업',
 '##을',
 '한다']

In [ ]:
# `hidden_states` has shape [13 x 1 x 22 x 768]
# `token_vecs` is a tensor with shape [22 x 768]
token_vecs = hidden_states[-2][0]

# Calculate the average of all 22 token vectors.
sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_kobert_sentvector(sent_A, 
                          to_get_cls:bool = False, 
                          start_idx_layer:int = None, 
                          end_idx_layer:int = None,
                          print_outputs_shapes = False):

    model.eval()
    model.to(device)
    
    tokens_pt = tokenizer(
            sent_A,
            max_length = tokenizer.model_max_length,
            return_tensors="pt",
            truncation = False,
            add_special_tokens = True,
            padding = 'max_length'
    )

    tokens_pt = {k: v.to(device) for k, v in tokens_pt.items()}

    with torch.no_grad():# 그라디엔트 계산 비활성화
        last_hidden_state, pooler_output, hidden_states, attentions = model(**tokens_pt, 
                                                                            output_hidden_states = True, 
                                                                            output_attentions=True,
                                                                            return_dict=False) # return_dict=True 시 outputs = model(**pt)로 return 받아야
    hidden_states, attentions = map(torch.cat, [hidden_states, attentions])

    if print_outputs_shapes:
        print('last_hidden_state.shape', last_hidden_state.shape)
        print('pooler_output.shape', pooler_output.shape)
        print('hidden_states.shape', hidden_states.shape)
        print('attentions.shape', attentions.shape)

    if to_get_cls:
        logits = last_hidden_state[:,0,:].detach().cpu().numpy()
    
    if not to_get_cls:
        assert isinstance(start_idx_layer, int) and isinstance(end_idx_layer, int), "set start and end layer index arguments"
        # hidden_states:tuple
        #logits = torch.cat(hidden_states)[:, start_idx_layer:end_idx_layer+1, :].detach().cpu().numpy()
        logits = hidden_states[:, start_idx_layer:end_idx_layer, :].detach().cpu().numpy()

    torch.cuda.empty_cache()

    return logits

# sep = chr(255)
# temp_li = train_article_data['text'].tolist()
# temp_doc = temp_li[0]
sent_A = '장갑을 낀 사람이 밭에 있다\n그 사람이 허리를 숙이고 있다\n그 사람이 작업을 한다'

layer_num = 13 # kobert layer num (+emb layer)

logits = get_kobert_sentvector(sent_A, to_get_cls = True)
print(logits.shape)